In [5]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Choose Object Category and Types
object_category = "Kitchen Utensils"
object_types = ["spoon", "fork", "butter knife", "cutting knife", "ladle"]

# Step 2: Collect Dataset
dataset_dir = "bow"  # Directory containing subfolders for each object type
image_paths = []
labels = []

for object_type in object_types:
    object_dir = os.path.join(dataset_dir, object_type)
    for filename in os.listdir(object_dir):
        if filename.endswith(".jpg"):
            image_path = os.path.join(object_dir, filename)
            image_paths.append(image_path)
            labels.append(object_type)

# Step 3: Preprocess Images
def preprocess_image(image_path, target_size=(100, 100)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image

# Step 4: Feature Extraction and Building Visual Vocabulary
def build_visual_vocabulary(image_paths, feature_extractor=cv2.SIFT_create(), vocabulary_size=50):
    bow_trainer = cv2.BOWKMeansTrainer(vocabulary_size)
    for image_path in image_paths:
        image = preprocess_image(image_path)
        keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
        if descriptors is not None:
            bow_trainer.add(descriptors)
    visual_vocabulary = bow_trainer.cluster()
    return visual_vocabulary

# Step 5: Feature Representation
def create_feature_histograms(image_paths, feature_extractor, visual_vocabulary):
    bow_extractor = cv2.BOWImgDescriptorExtractor(feature_extractor, cv2.BFMatcher(cv2.NORM_L2))
    bow_extractor.setVocabulary(visual_vocabulary)
    histograms = []
    for image_path in image_paths:
        image = preprocess_image(image_path)
        bow_descriptor = bow_extractor.compute(image, feature_extractor.detect(image))
        histograms.append(bow_descriptor)
    return np.array(histograms)

# Step 6: Train Classifier
def train_classifier(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    classifier = SVC(kernel='linear')
    classifier.fit(X_train, y_train)
    return classifier, X_test, y_test

# Step 7: Evaluate Performance
def evaluate_classifier(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("Classification Report:\n", report)

# Main Function
def main():
    # Step 4: Feature Extraction and Building Visual Vocabulary
    feature_extractor = cv2.SIFT_create()
    visual_vocabulary = build_visual_vocabulary(image_paths, feature_extractor)

    # Step 5: Feature Representation
    features = create_feature_histograms(image_paths, feature_extractor, visual_vocabulary)

    # Step 6: Train Classifier
    classifier, X_test, y_test = train_classifier(features, labels)

    # Step 7: Evaluate Performance
    evaluate_classifier(classifier, X_test, y_test)

if __name__ == "__main__":
    main()


Objects detected in frame_0028.jpg and saved at detected_objects/detected_objects_frame_0028.jpg
Objects detected in frame_0014.jpg and saved at detected_objects/detected_objects_frame_0014.jpg
Objects detected in frame_0000.jpg and saved at detected_objects/detected_objects_frame_0000.jpg
Objects detected in frame_0216.jpg and saved at detected_objects/detected_objects_frame_0216.jpg
Objects detected in frame_0202.jpg and saved at detected_objects/detected_objects_frame_0202.jpg
Objects detected in frame_0148.jpg and saved at detected_objects/detected_objects_frame_0148.jpg
Objects detected in frame_0160.jpg and saved at detected_objects/detected_objects_frame_0160.jpg
Objects detected in frame_0174.jpg and saved at detected_objects/detected_objects_frame_0174.jpg
Objects detected in frame_0175.jpg and saved at detected_objects/detected_objects_frame_0175.jpg
Objects detected in frame_0161.jpg and saved at detected_objects/detected_objects_frame_0161.jpg
Objects detected in frame_0149